# GPT From Scratch

Based on theory from the paper titled ['Attention is all you need'](https://arxiv.org/pdf/1706.03762) & a Youtube video named [
Let's build GPT: from scratch, in code, spelled out](https://www.youtube.com/watch?v=kCc8FmEb1nY&t=1221s) by Andrej Karpathy.

The **dataset** used in this project : [Tiny Shakespeare](https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt) (the totality of all the words written by Shakespeare).

The **goal** of this model is predicting the next token (subword in our case) based on the given context.

In [79]:
# all the necessary imports in one place
import wget

import numpy as np

import torch
import torch.nn as nn
from torch.nn import functional as F

In [80]:
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
wget.download(url)

100% [......................................................] 1115394 / 1115394

'input (2).txt'

## Importing and Preprocessing necessary data

In [81]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [82]:
print(text[:300])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [83]:
print('Length of dataset in characters:', len(text))

Length of dataset in characters: 1115394


Let's obtain all the unique characters in our dataset (sorted) <br>
(when printing the ' 's were placed to see that the first element is a blank space character, which is also considered a character in the vocabulary)

### Tokenization

To keep things simple we are using a character-level tokenizer. But real GPTs mainly use more advanced tokenizers such as BPE

In [84]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print('All the characters present in the dataset: ', "' '".join(chars))
print('Vocabulary Length : ', len(chars))

All the characters present in the dataset:  
' ' ' '!' '$' '&' ''' ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z
Vocabulary Length :  65


### Vectorization

In this section we find a way to turn our string into an integer representation, and we define 2 crucial components of the GPT:
- Encoder : takes a string and outputs a list of integers
- Decoder : takes a list of integers and outputs corresponding string

In [85]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

encoder = lambda seq: [stoi[char] for char in seq]
decoder = lambda int_list: ''.join([itos[i] for i in int_list])

In [86]:
print(encoder('hello there'))
print(decoder([46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
hello there


Lets apply this to the entire dataset:

In [87]:
data = torch.tensor(encoder(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


Let's split the data into train and validation sets:
- 90% train
- 10% validation

In [88]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

Key point: we are not going to feed all the data to a transformer at one, since it would be too computationally expensive. We sample random chunks of data that have a max length (block_size) and work with them.

In [89]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

Given a block_size the model will have to make 8 predictions, so in our example: <br>
given 18, predict 47, <br>
given 18, 47, predict 56 etc. <br>

In [90]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size): # t for time dimension
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context} the target: {target}')

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


The subtility of creating the context in this way is making the transformer see not only the last word and the target, but also the words that came before it, forming a context.<br><br>
We will also have batches of certain dimensions containing multiple chunks of data.

In [91]:
torch.manual_seed(1337)
BATCH_SIZE = 4
BLOCK_SIZE = 8

In [92]:
def get_batch(split):
    # generate a batch of data of inputs x and target y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE, ))
    x = torch.stack([data[i:i + BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i + 1:i + BLOCK_SIZE + 1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:', xb.shape)
print(xb)
print('targets:', yb.shape)
print(yb)

inputs: torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [93]:
for b in range(BATCH_SIZE):
    for t in range(BLOCK_SIZE):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target is {target}")

when input is [24] the target is 43
when input is [24, 43] the target is 58
when input is [24, 43, 58] the target is 5
when input is [24, 43, 58, 5] the target is 57
when input is [24, 43, 58, 5, 57] the target is 1
when input is [24, 43, 58, 5, 57, 1] the target is 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
when input is [44] the target is 53
when input is [44, 53] the target is 56
when input is [44, 53, 56] the target is 1
when input is [44, 53, 56, 1] the target is 58
when input is [44, 53, 56, 1, 58] the target is 46
when input is [44, 53, 56, 1, 58, 46] the target is 39
when input is [44, 53, 56, 1, 58, 46, 39] the target is 58
when input is [44, 53, 56, 1, 58, 46, 39, 58] the target is 1
when input is [52] the target is 58
when input is [52, 58] the target is 1
when input is [52, 58, 1] the target is 58
when input is [52, 58, 1, 58] the target is 46
when input is [52, 58, 1, 58, 46] the target is 39
w

Before creating the GPT, we will first try the dataset on lighter models.

#### Bigram Model

We'll start with the Bigram model, which predicts the next word based on the previous one.

In [119]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token reads off the logits for the next token directly from the lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensors of integers
        logits = self.token_embedding_table(idx) # scores for next character in sequence of size (B, T, C)

        if targets is None:
            loss = None
        else:
            # modifications of shape since cross_entropy expects to see another shape
            B, T, C = logits.shape
            logits = logits.view(B * T, C) 
            targets = targets.view(B * T)
            
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            logits = logits[:, -1, :] # make it (B, C)
            probs = F.softmax(logits, dim=-1) # applying softmax to get probas
            idx_next = torch.multinomial(probs, num_samples=1) # shape (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)
            
        return idx  

In [120]:
bigram_model = BigramLanguageModel(vocab_size)
logits, loss = bigram_model(xb, yb)

print(logits.shape)
print(loss)

print(decoder(bigram_model.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.5330, grad_fn=<NllLossBackward0>)

Jx:E$;
'ana:.p&mY&3Ww3GRMCyRGmD$SLjtLj AZwuoF3WJYQyqxIAS-Mu&RHeaTwWPfiS
gti3dcpNd be:R;XvyNe3YChSL!Z


As we can see the predictions are far from being something even related to text we can understand, let alone Shakespeare. Let's add an optimizer first:

In [125]:
optimizer = torch.optim.AdamW(bigram_model.parameters(), lr=1e-3)

Next we will see a typical training setup of 100 iterations on PyTorch:

In [132]:
batch_size = 32 # let's pick a bigger batch size

for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = bigram_model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4513444900512695


In [134]:
print(decoder(bigram_model.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist()))


Boemanse oue bethefin,
HNept,

CIZx-FRoke t;
INI h-e the w'f ce?nthtur
This ianspy GOMan y s g.
R s Dowouth r areny mern met
I is f hait heard LI has ourjNy wes:
aimede in whe wndeildr mad:

Aly BEdy
Ord is RY: tof ath bast:


Taif t amy co.
FRI ans; tanme:
NCIXRI my the ts.
PEY ce hecive If T:
Itit


As we can see, the model has gone through some improvement. However, we can clearly see that we are far from being able to generate Shakespeare.

Up until this point, we have never used the **context** of the text to make predictions, since the model was too simple. It is important not to look at the 'future' words, so we only consider the (average) context that comes previously:

## The Mathematical Trick in Self Attention

Let's look at a toy example :

In [138]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

### Version 1

In [150]:
x_bow = torch.zeros((B, T, C))

for batch in range(B):
    for t in range(T):
        x_prev = x[batch, :t+1] # (t, C)
        x_bow[batch, t] = torch.mean(x_prev, 0) # averaging out the time

This is how we get an average of the context that came before. However, there is a better solution by multiplying normalized matrices.

### Version 2

In [156]:
wei = torch.tril(torch.ones(T, T)) # tril() creates a lower triangular matrix
wei = wei / wei.sum(1, keepdim=True) # normalizing the matrix to get the average after matrix mult
x_bow2 = wei @ x # (B, T, T) @ (B, T, C) --> (B, T, C)
torch.allclose(x_bow, x_bow2)

True

Another version consists in using SoftMax to get the right matrix:

 ### Version 3

In [161]:
tril = torch.tril(torch.ones(T, T))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [168]:
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf')) # if tril == 0 replace with -inf
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [169]:
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [170]:
x_bow3 = wei @ x
torch.allclose(x_bow, x_bow3)

True

So the takeaway from this part of the notebook is that we can do weighted aggregations of the past elements by doing matrix multiplications on a lower triangular matrix. 

## Self-Attention Mechanism

Each token in the data has 2 representations : Query and Key.
We calculate the dot product of each Query word with Key words. If the words are 'similar', we will get a high score and we will get to learn more about that specific token as opposed to any other token in the sequence.
<br>
In most real-world models multiple heads are used (independent computations of attention) so that we are able to capture **different relationships in different heads**.

The value is the agregated result.

The self-attention mechanism is computed as:

$$
\text{Attention}(Q, K, V) = \text{softmax}\Big(\frac{Q K^T}{\sqrt{d_k}}\Big) V
$$

the sqrt(dk) (where dk is head_size) is an important normalization step to preserve the initial variance. Since wei will then be fed into a Softmax function, it is important for wei to be **fairly diffused**.

If wei takes on very positive or very negative numbers inside it, Softmax might converge into one-hot vectors, and that is not what we want.

In [197]:
torch.manual_seed(1337)

B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16 # hyperparameter
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) * head_size ** -0.5 # similarity matrix
# (B, T, 16) @ (B, 16, T) ---> (B, T, T)
tril = torch.tril(torch.ones(T, T))

# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1) # last dimension
v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [198]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3966, 0.6034, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3069, 0.2892, 0.4039, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3233, 0.2175, 0.2443, 0.2149, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1479, 0.2034, 0.1663, 0.1455, 0.3369, 0.0000, 0.0000, 0.0000],
        [0.1259, 0.2490, 0.1324, 0.1062, 0.3141, 0.0724, 0.0000, 0.0000],
        [0.1598, 0.1990, 0.1140, 0.1125, 0.1418, 0.1669, 0.1061, 0.0000],
        [0.0845, 0.1197, 0.1078, 0.1537, 0.1086, 0.1146, 0.1558, 0.1553]],
       grad_fn=<SelectBackward0>)

So in this example (last row) we can see that the element in the 3rd position was interesing to the last element since it has the highest weight.

### Some Useful Notes on Attention:
- Attention is a **communication mechanism**.
- There is no **notion of space**. That is why we have to positionally encode tokens to make use of this information.
- Each example across batch dimension is processed **completely independently**.
- In some use cases it is ok to look at the words that come in the future, e.g. Sentiment Analysis. It is then called an Encoder block.

Full trainable models (bigram.py and gpt.py) can be found in the GitHub repository this Notebook is Uploaded on.